# 00 — Check GPU and sampling backends

This notebook helps you verify that `gpu-profile` can read utilization metrics in your environment.

It will:

- show `nvidia-smi` output (if available)
- run a short `GpuMonitor` block
- optionally compare NVML vs `nvidia-smi` backends

> Tip: for the lowest overhead sampling, install NVML support:
>
> ```bash
> pip install gpu-profile[nvml]
> ```


In [ ]:
import shutil, subprocess, textwrap

print('nvidia-smi on PATH:', shutil.which('nvidia-smi') is not None)
if shutil.which('nvidia-smi'):
    out = subprocess.check_output(['nvidia-smi'], text=True)
    print(out)
else:
    print('nvidia-smi not found; NVML may still work if drivers expose it and you installed nvidia-ml-py3.')


In [ ]:
import time
from gpu_profile import GpuMonitor

# strict=False means: if no backend is available, we get a summary with notes instead of an exception.
with GpuMonitor(interval_s=0.5, strict=False) as mon:
    time.sleep(3)

print(mon.summary.format())


In [ ]:
import time
from gpu_profile import GpuMonitor, GpuBackendError

for backend in ['nvml', 'smi']:
    print()
    print('--- backend:', backend, '---')
    try:
        with GpuMonitor(backend=backend, interval_s=0.5, strict=True) as mon:
            time.sleep(2)
        print(mon.summary.format())
    except GpuBackendError as e:
        print('backend not available:', e)
